In [1]:
%cd ../..

/home/kirill/PycharmProjects/pythonProject/RecoServiceTemplate


In [ ]:
import pandas as pd
from implicit.nearest_neighbours import CosineRecommender
from rectools.dataset import Dataset
from rectools import Columns
from rectools.models.popular import PopularModel
from models.userknn.userknn import UserKnn
from rectools.model_selection import TimeRangeSplit
from models.config import UserKnn_model, Popular_model

In [8]:
# Train model flag
RETRAIN = False

# Get KION dataset

In [9]:
interactions = pd.read_csv('{0}/interactions.csv'.format(UserKnn_model.dataset_path))
users = pd.read_csv('{0}/users.csv'.format(UserKnn_model.dataset_path))
items = pd.read_csv('{0}/items.csv'.format(UserKnn_model.dataset_path))

In [10]:
# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight},
                    inplace=True)

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

In [11]:
dataset = Dataset.construct(
            interactions_df=interactions,
            user_features_df=None,
            item_features_df=None,
        )

# Fit Popular model

In [12]:
pop = PopularModel()
pop.fit(dataset)

# Get recomendations

In [15]:
popular_reco_df = pop.recommend(
    dataset.user_id_map.external_ids[:1],
    dataset=dataset,
    k=Popular_model.N_recs,
    filter_viewed=False  # True - throw away some items for each user
)

,user_id,item_id,score,rank
0,0,10440,202457.0,1
1,0,15297,193123.0,2
2,0,9728,132865.0,3
3,0,13865,122119.0,4
4,0,4151,91167.0,5
5,0,3734,74803.0,6
6,0,2657,68581.0,7
7,0,4880,55043.0,8
8,0,142,45367.0,9
9,0,6809,40372.0,10


# Save recomendations df

In [14]:
popular_reco_df.to_csv(Popular_model.save_reco_df_path,
                       encoding='utf-8', index=False)